In [12]:
import pandas as pd
import numpy as np
import json
from datetime import datetime
from sklearn.preprocessing import LabelEncoder
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack

In [13]:
with open('/kaggle/input/vietnamese-online-news-dataset/news_dataset.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

df = pd.DataFrame(data)
df

,id,author,content,picture_count,processed,source,title,topic,url,crawled_at
0,218270,,"Chiều 31/7, Công an tỉnh Thừa Thiên - Huế đã c...",3,0,docbao.vn,"Tên cướp tiệm vàng tại Huế là đại uý công an, ...",Pháp luật,https://docbao.vn/phap-luat/ten-cuop-tiem-vang...,2022-08-01 09:09:22.817308
1,218269,(Nguồn: Sina),"Gần đây, Thứ trưởng Bộ Phát triển Kỹ thuật số,...",1,0,vtc.vn,"Bỏ qua mạng 5G, Nga tiến thẳng từ 4G lên 6G",Sống kết nối,https://vtc.vn/bo-qua-mang-5g-nga-tien-thang-t...,2022-08-01 09:09:21.181469
2,218268,Hồ Sỹ Anh,Kết quả thi tốt nghiệp THPT năm 2022 cho thấy ...,3,0,thanhnien.vn,Địa phương nào đứng đầu cả nước tổng điểm 3 mô...,Giáo dục,https://thanhnien.vn/dia-phuong-nao-dung-dau-c...,2022-08-01 09:09:15.311901
3,218267,Ngọc Ánh,Thống đốc Kentucky Andy Beshear hôm 31/7 cho h...,1,0,vnexpress,Người chết trong mưa lũ 'nghìn năm có một' ở M...,Thế giới,https://vnexpress.net/nguoi-chet-trong-mua-lu-...,2022-08-01 09:09:02.211498
4,218266,HẢI YẾN - MINH LÝ,Vụ tai nạn giao thông liên hoàn trên phố đi bộ...,12,0,soha,"Hải Phòng: Hình ảnh xe ""điên"" gây tai nạn liên...",Thời sự - Xã hội,https://soha.vn/hai-phong-hinh-anh-xe-dien-gay...,2022-08-01 09:09:01.601170
...,...,...,...,...,...,...,...,...,...,...
184534,5,Quốc Thắng,"Chiều 7/6, Khoa bị Công an quận Phú Nhuận phối...",2,1,vnexpress,Nghi can sát hại cha bị bắt,Pháp luật,https://vnexpress.net/nghi-can-sat-hai-cha-bi-...,2022-06-07 14:05:53.848085
184535,4,Thảo Mi,"Ngày 7/6, phiên xử phúc thẩm ông Tất Thành Can...",2,1,vnexpress,Ông Tất Thành Cang được đề nghị giảm án,Pháp luật,https://vnexpress.net/ong-tat-thanh-cang-duoc-...,2022-06-07 14:05:53.482540
184536,3,Thanh Tâm,"""Các khu dân cư của thành phố Severodonetsk đã...",2,1,vnexpress,Nga tuyên bố 'giải phóng' hoàn toàn khu dân cư...,Thế giới,https://vnexpress.net/nga-tuyen-bo-giai-phong-...,2022-06-07 14:05:51.797545
184537,2,Lâm Thỏa,"*Việt Nam - Malaysia: 20h thứ Tư 8/6, trên VnE...",2,1,vnexpress,HLV Malaysia dọa ngáng đường Việt Nam,Thể thao,https://vnexpress.net/hlv-malaysia-doa-ngang-d...,2022-06-07 14:05:51.433712


In [14]:
vietnamese_stopwords = set([
    'bị', 'bởi', 'cả', 'các', 'cái', 'cần', 'càng', 'chỉ', 'chiếc', 'cho', 'chứ', 'chưa', 'chuyện', 
    'có', 'có thể', 'cứ', 'của', 'cùng', 'cũng', 'đã', 'đang', 'đây', 'để', 'đến nỗi', 'đều',
    'điều', 'do', 'đó', 'được', 'dưới', 'gì', 'khi', 'không', 'là', 'lại', 'lên', 'lúc', 'mà',
    'mỗi', 'một cách', 'này', 'nên', 'nếu', 'ngay', 'nhiều', 'như', 'nhưng', 'những', 'nơi', 
    'nữa', 'phải', 'qua', 'ra', 'rằng', 'rất', 'rồi', 'sau', 'sẽ', 'so', 'sự', 'tại', 'theo', 
    'thì', 'trên', 'trước', 'từ', 'từng', 'và', 'vẫn', 'vào', 'vậy', 'vì', 'việc', 'với', 'vừa'
])

In [15]:
df.drop(columns=['id', 'processed'], inplace=True)

In [16]:
# Convert to lowercase and remove leading/trailing whitespace
df['author'] = df['author'].str.lower().str.strip()
df['content'] = df['content'].str.lower().str.strip()
df['title'] = df['title'].str.lower().str.strip()
df['topic'] = df['topic'].str.lower().str.strip()

In [17]:
# Remove Vietnamese stopwords
def remove_vietnamese_stopwords(text):
    words = re.findall(r'\b\w+\b', text)  
    return ' '.join([word for word in words if word not in vietnamese_stopwords])

df['content'] = df['content'].apply(remove_vietnamese_stopwords)

In [18]:
# Label encoding for 'topic' and 'source'
source_label_encoder = LabelEncoder()
df["source"] = source_label_encoder.fit_transform(df["source"])
topic_label_encoder = LabelEncoder()
df["topic"] = topic_label_encoder.fit_transform(df["topic"])

In [19]:
df

,author,content,picture_count,source,title,topic,url,crawled_at
0,,chiều 31 7 công an tỉnh thừa thiên huế thông t...,3,8,"tên cướp tiệm vàng tại huế là đại uý công an, ...",944,https://docbao.vn/phap-luat/ten-cuop-tiem-vang...,2022-08-01 09:09:22.817308
1,(nguồn: sina),gần thứ trưởng bộ phát triển kỹ thuật số truyề...,1,22,"bỏ qua mạng 5g, nga tiến thẳng từ 4g lên 6g",1057,https://vtc.vn/bo-qua-mang-5g-nga-tien-thang-t...,2022-08-01 09:09:21.181469
2,hồ sỹ anh,kết quả thi tốt nghiệp thpt năm 2022 thấy điểm...,3,15,địa phương nào đứng đầu cả nước tổng điểm 3 mô...,447,https://thanhnien.vn/dia-phuong-nao-dung-dau-c...,2022-08-01 09:09:15.311901
3,ngọc ánh,thống đốc kentucky andy beshear hôm 31 7 hay đ...,1,20,người chết trong mưa lũ 'nghìn năm có một' ở m...,1113,https://vnexpress.net/nguoi-chet-trong-mua-lu-...,2022-08-01 09:09:02.211498
4,hải yến - minh lý,vụ tai nạn giao thông liên hoàn phố đi bộ tam ...,12,14,"hải phòng: hình ảnh xe ""điên"" gây tai nạn liên...",1124,https://soha.vn/hai-phong-hinh-anh-xe-dien-gay...,2022-08-01 09:09:01.601170
...,...,...,...,...,...,...,...,...
184534,quốc thắng,chiều 7 6 khoa công an quận phú nhuận phối hợp...,2,20,nghi can sát hại cha bị bắt,944,https://vnexpress.net/nghi-can-sat-hai-cha-bi-...,2022-06-07 14:05:53.848085
184535,thảo mi,ngày 7 6 phiên xử phúc thẩm ông tất thành cang...,2,20,ông tất thành cang được đề nghị giảm án,944,https://vnexpress.net/ong-tat-thanh-cang-duoc-...,2022-06-07 14:05:53.482540
184536,thanh tâm,khu dân cư thành phố severodonetsk hoàn toàn g...,2,20,nga tuyên bố 'giải phóng' hoàn toàn khu dân cư...,1113,https://vnexpress.net/nga-tuyen-bo-giai-phong-...,2022-06-07 14:05:51.797545
184537,lâm thỏa,việt nam malaysia 20h thứ tư 8 6 vnexpress mal...,2,20,hlv malaysia dọa ngáng đường việt nam,1118,https://vnexpress.net/hlv-malaysia-doa-ngang-d...,2022-06-07 14:05:51.433712


In [21]:
df.to_csv('preprocessed_data.csv', index=False)